In [30]:
import numpy as np
import random
import sys
from PIL import Image
import os # ,cv2

from matplotlib import pyplot as plt


In [31]:

data=[]
dataset=os.listdir('dataset')
for class_num,data_class in enumerate(dataset):
    print(f"Loading class: {data_class}")
    directory=f'dataset/{data_class}/'
    class_files=[os.path.join(directory, file) for file in os.listdir(directory)]
    for file in class_files:
        img = Image.open(file).convert('L')
        img=np.array(img) #/ 255
        dataWithLabel=np.append(img,class_num)
        data.append(dataWithLabel)
    

Loading class: 10
Loading class: 50
Loading class: 400
Loading class: 900
Loading class: 40
Loading class: 90
Loading class: 500
Loading class: 100
Loading class: 1000
Loading class: 5
Loading class: 1
Loading class: 9
Loading class: 4


In [32]:
data = np.array(data)
dataset_size, _ = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets
train_amount=int(dataset_size*0.8)

data_train = data[0:train_amount].T
X_train = data_train[0:-1]
X_train = X_train / 255
Y_train = data_train[-1]
_,m_train = X_train.shape

data_test = data[train_amount:].T
X_test = data_test[0:-1]
X_test = X_test / 255
Y_test = data_test[-1]


X_train=(X_train.T).flatten().reshape(1040,500,500)
X_train=(X_test.T).flatten().reshape(260,500,500)

In [33]:
Y_train

array([ 8,  7,  4, ...,  0,  5, 12])

In [46]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def ReLU_deriv(Z):
    return Z > 0

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y
def convolve(image,kernel):
    k_x,k_y=kernel.shape
    x_size=image.shape[0]-k_x
    y_size=image.shape[1]-k_y
    out=np.zeros((x_size,y_size))
    for x in range(x_size):
        for y in range(y_size):
            out[x][y]=np.tensordot(image[x:x+k_x,y:y+k_y],kernel)

In [34]:
class Convolutional:
        def __init__(self,  kernel_specs=(3,3), padding=1, stride=1, bias=0.1, eta=0.01): #eta=learning rate
                self.specs=kernel_specs               
                self.padding=padding
                self.stride=stride
                self.eta = eta
                self.iteration = 0                       
                self.cache = 0                  
                self.kernel=np.random.rand(*kernel_specs)*0.1

        def pad(self,images):
            new_images=[]
            for image in images:
                p=self.padding
                x,y=image.shape
                new_img=np.zeros((x+p, y+p))
                new_img[p:,p:]=image
                new_images.append(new_img)
            return new_images
        
        def forward_prop(self, images):
            if(self.padding!=0):
                images=self.pad(images)
            self.cache=images
            out=[]
            for image in images:
                out.append(convolve(image,np.flip(self.kernel)))
            #out=[]
            #for kernel in self.filters:
            #    out.append(convolve(image,kernel))
            out=np.array(out)
            return out
        def backward_prop(self, d_out):
            flipped_kernel=np.flip(self.kernel)
            padded_d_out=pad(d_out,flipped_kernel.shape[0]-1)
            d_input=convolve(padded_d_out,flipped_kernel)
            d_kernel=convolve(self.cache,np.flip(d_out)) #*self.dW))
            """d_input=[]
            d_filters=[]
            for kernel in self.filters:
                kernel=np.flip(kernel)
                d_input.append(convolve(d_out,kernel))
                d_filters.append(d_out)
            d_input=np.array(d_input)
            d_filter=[]
            """
            return d_input

        def gd(self, d_kernel): #, d_bias):
                    self.eta = self.eta * np.exp(-self.iteration/20000)
                    self.kernel -= self.eta * (d_kernel)
                    self.iteration +=1

In [35]:
class Pooling: 
        def __init__(self, image_dim=(1, 16,16), mode='avg', size=2, stride=2):
                self.cache=None
                #self.image_dim=image_dim
                self.size=size
                self.stride=stride
                self.mode=mode
        def forward_prop(self,image): #For Pooling layers, it is not common to pad the input using zero-padding.
                w_in, h_in = self.image_dim.shape

                w_out = int((w_in - self.size)/self.stride)+1
                h_out = int((h_in - self.size)/self.stride)+1

                out=np.zeros((layers, w_out, h_out))

                #for layer in range(layers):
                y_out, x_out = 0,0
                for y in range(0, h_in - self.size, self.stride):
                        x_out=0
                        for x in range(0, w_in - self.size, self.stride):
                                if self.mode=='avg':
                                        out[ y_out, x_out] = np.average(images[  y:y+self.size, x:x+self.size])
                                else: #max pooling is applied
                                        out[layer, y_out, x_out] = np.max(images[  y:y+self.size, x:x+self.size])
                                x_out+=1
                        y_out+=1
                return out


        def backward_prop(self, d_out): #d_out is like the derivative of the pooling output, image is the input of the pooling layer
                
            layers, w_in, h_in = self.image_dim

            w_out = int((w_in - self.size)/self.stride)+1
            h_out = int((h_in - self.size)/self.stride)+1

            out = np.zeros((layers, w_in, h_in))

            y_out, x_out = 0,0
            for y in range(0, h_out, self.stride):
                x_out=0

                for x in range(0, w_out, self.stride):

                    if self.mode=='avg':    #not sure about that

                        average_dout=d_out[y_out,x_out]/(self.size*2)
                        out[ y:(y+self.size), x:(x+self.size)] += np.ones((self.size,self.size))*average_dout

                    else: #max pooling is applied

                        area = self.cache[ y:y+self.size, x:x+self.size]
                        index = np.nanargmax(area)
                        (y_i, x_i) = np.unravel_index(index, area.shape)
                        out[ y + y_i, x + x_i] += d_out[ y_out, x_out]
                    x_out+=1
                y_out+=1
                return  out

In [36]:
class Dense:
    def forward_prop(X, W, b, activation):
        Z = W.dot(X) + b
        A = activation(Z)
        return Z,A

    def backward_prop(dZ, A):
        dW = 1 / dataset_size * dZ.dot(A.T)
        db = 1 / dataset_size * np.sum(dZ)
        return dW, db

    def get_delta(dZ, Z_prev, W, deriv):
        dZ_previous = W.T.dot(dZ) * deriv(Z_prev)
        return dZ_previous
    
    def update_params(dW, db, W, b, eta):
        W = W - alpha * dW
        b = b - alpha * db   
        return W, b

In [38]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size


In [44]:
eta=0.01
epochs=100
C1_layer=Convolutional()
P1_layer=Pooling()
C2_layer=Convolutional()
P2_layer=Pooling(mode='max')
W1,b1,W2,b2 = init_params()
X_train=X_train.flatten()
for iteration in range(epochs):
    print(iteration)
    #C1_out = C1_layer.forward_prop(X_train)
    #P1_out = P1_layer.forward_prop(C1_out)
    #print("C2")
    #C2_out = C2_layer.forward_prop(P1_out)
    #P2_out = P2_layer.forward_prop(C2_out)
    #convolution_flatten = P2_out.flatten()
    Z1, A1 = Dense.forward_prop(X_train,W1,b1,ReLU)
    Z2, A2 = Dense.forward_prop(A1,W2,b2,softmax)

    dZ2 = A2-onehot_Y
    dW2, db2 = Dense.backward_prop(dZ2,A1)
    dZ1 = Dense.get_delta(dZ2, Z1, W2, ReLU_deriv)
    dW1, db1 = Dense.backward_prop(dZ2,X_train)

    eta=eta * np.exp(-iteration/20000)
    W1,b1 = Dense.update_params(dW1,db1,W1,b1,eta)
    W2,b2 = Dense.update_params(dW2,db2,W2,b2,eta)

0


ValueError: shapes (10,784) and (260,500,500) not aligned: 784 (dim 1) != 500 (dim 1)

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

In [7]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [45]:
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)
test_prediction(2, W1, b1, W2, b2)
test_prediction(3, W1, b1, W2, b2)

NameError: name 'test_prediction' is not defined